Dodawanie nowego bloku do blockchainu:
1. Zapisanie danych, jako transakcji w odpowiedniej zmiennej.
2. Stworzenie blockchainu, czyli pustej listy, do której będziemy dodawać kolejne bloki.
3. Sprawdzenie czy liczba transakcji dla danego bloku wynosi sześć.
4. Dodanie do blockchainu pierwszego bloku, czyli Bloku Genesis, zawierającego zmienne index, data, timestamp, previous_hash, nonce.
5. Znalezienie odpowiedniego hasha SHA-256, który pasowałby do określonego schematu i ustalonej na początku trudności szukania takiego hasha.
6. Po znalezieniu odpowiedniej wartości "nonce", dodajemy Blok Genesis do blockchainu, wyświetlając przy tym odpowiedni komunikat.
7. Taki sam mechanizm jest stosowany podczas dodawania kolejnych bloków, jeżeli blok będzie mógł pobrać przynajmniej sześć transakcji. Zmienia się jedynie indeks bloku o jeden wyżej oraz zmienna "previous_hash" jest ustawiana na wyliczony wcześniej hash poprzedniego bloku w blockchainie.
8. Na koniec, wszystkie bloki w blokchainie zostają wyświetlone oraz blockchain zostaje odpowiednio sprawdzony. W przypadku, gdy wystąpiła podmiana wartości, w którymś bloku, funkcja sprawdzająca ich poprawność zwróci wartość "False".

Dodanie podstawowych bibliotek:<br>
> hashlib - hashowanie bloku za pomocą SHA-256<br>
time - pobieranie czasu rzeczywistego<br>
re - tworzenie wyrażeń regularnych<br>
termcolor - wyświetlania danych wyjściowych w kolorze<br>

In [ ]:
import sys
!{sys.executable} -m pip install colored

In [ ]:
import hashlib
import time
import re
from termcolor import colored

Poniższy kod zawiera wyświetlenie w odpowiednim formacie całego blockainu. Dodatkowo dane wyjściowe są wyświetlane w kolorze, co poprawia ich czytelność.
<br>
<br>
**print_blockchain()** - przyjmuje, jako argument łańcuch bloków, a następnie wyświetla każdy blok, używając funkcji pprint()

In [ ]:
def print_blockchain(blockchain):
    for block in blockchain:
        print(colored(f"Block {block}", "red"))
        pprint(block)


def pprint(block):
    print('{')
    print("\t", f"{colored('index', 'green')}: {block.index}")
    print("\t", f"{colored('data', 'green')}: {block.data}")
    print("\t", f"{colored('timestamp', 'green')}: {block.timestamp}")
    print("\t", f"{colored('previous_hash', 'green')}: {block.previous_hash}")
    print("\t", f"{colored('nonce', 'green')}: {block.nonce}")
    print("\t", f"{colored('hash', 'green')}: {block.hash}")
    print('}')
    print()

**validating()** - przyjmuje, jako argument łańcuch bloków i sprawdza, czy każdy blok ma poprawny hash i czy hash poprzedniego bloku się zgadza. Zwraca True ("Blockchain is valid") jeśli łańcuch jest poprawny, w przeciwnym razie zwraca False ("Blockchain is not valid").

In [ ]:
def validating(blockchain):
    # If only blockGenesis
    if (len(blockchain) == 1):
        return True
    for i in range(1, len(blockchain)):
        current_block = blockchain[i]
        previous_block = blockchain[i - 1]
        if current_block.hash != current_block.calc_hash() or previous_block.hash != current_block.previous_hash:
            return "Blockchain is not valid!"
    return "Blockchain is valid"

**hex_to_bin()** - przyjmuje, jako argument liczbę w postaci szesnastkowej i zwraca tę liczbę w postaci binarnej, pozostawiając odpowiednią liczbę zer na początku.

In [ ]:
def hex_to_bin(hex):
    return (bin(int(hex, 16))[2:]).zfill(len(hex) * 4)

**read_data()** - przyjmuje, jako argument ścieżkę do pliku tekstowego, gdzie podane są przykładowe transakcje, a następnie odczytuje te dane, usuwa z nich znaki nowej linii i dzieli je na grupy. W każdej grupie znajduje się sześć transakcji. Na koniec grupy te zostają przez funkcję zwrócone.

In [ ]:
def read_data(file_path):
    with open(file_path) as file:
        data = file.readlines()
        data = [line.rstrip('\n') for line in data]
        groups = [data[n:n+6] for n in range(0, len(data), 6)]
        # print(groups)
        return groups

*Klasa "Block" zawiera w sobie zmienną "difficulty", która powoduje ustawienie odpowiedniej trudności znajdowania hasha, przez zmianę liczby potrzebnych zer na początku tego hasha.*<br><br>
Konstruktor **__init__()** zawiera poniższe zmienne:<br>
**index** - numer identyfikatora bloku <br>
**data** - dane sześciu pobranych transakcji <br>
**timestamp** - czas dodania bloku do łańcucha<br>
**previous_hash** - hash poprzedniego bloku<br>
**nounce** (ang. number only used once) - pomaga znaleźć odpowiedni hash, który spełni ograniczenia poziomu trudności<br>
**hash** - hash aktualnego bloku)<br>
<br> 
**calc_hash()** - hashuje wszystkie zmienne znajdujące się w bloku za pomocą funkcji skrótu SHA-256, a na koniec zwraca hash w postaci heksadecymalnej.
<br><br>
**mining()** - odpowiada za proces szukania(kopania) hasha, czyli szukania takiego hasha, który spełni odpowiednie parametry. W tym przypadku, odpowiednią liczbę zer na początku hasha. Jeżeli nie spełnia, to zmienna "nonce" jest zwiększana o jeden.
<br><br>
**genesis_block(data)** - tworzy klasę "Block" o indeksie zero, a następnie czeka na "wykopanie". Jeżeli odpowiedni hash zostanie wyliczony, to wyświetlany jest komunikat o dodaniu bloku genesis. Hash poprzedniego bloku jest równy 64 zera.
<br><br>
**create_block(last_block, data), add_block(last_block, data)** - działają w podobny sposób, jak funkcja "genesis_block". Różnica polega jedynia na tym, że indeks jest zwiększany o jeden w porównaniu do poprzedniego bloku oraz potrzebny funkcji jest hash bloku poprzedniego.

In [ ]:
class Block:
    difficulty = 10

    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.data = data
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calc_hash()

    def calc_hash(self):
        sha = hashlib.sha256()
        sha.update((str(self.index) + str(self.timestamp) +
                   str(self.data) + str(self.previous_hash) + str(self.nonce)).encode())
        return sha.hexdigest()

    def mining(self):
        # regex - set the number of zeros at the beginning and zero or more numbers after it
        regex = "^(0){" + str(Block.difficulty) + "}.*"
        # convert hexadecimal hash to binary form
        binary_hash = hex_to_bin(self.hash)
        while not re.match(regex, binary_hash):
            # Increase the nonce value until we have the hash which will match the regex pattern
            self.nonce += 1
            self.hash = self.calc_hash()
            binary_hash = hex_to_bin(self.hash)
        return True


    def genesis_block(data):
        block = Block(0, time.time(), data, "0000000000000000000000000000000000000000000000000000000000000000")
        if block.mining():
            print(
                colored(f"Genesis block ID:{block.index} was added successfully!", "blue"))
            return block

    def create_block(last_block, data):
        index = last_block.index + 1
        previous_hash = last_block.hash
        return Block(index, time.time(), data, previous_hash)

    def add_block(last_block, data):
        block = Block.create_block(last_block, data)
        if block.mining():
            print(
                colored(f"Block ID:{block.index} was added successfully!", "blue"))
            return block

**create_blockchain()** - funkcja najpierw pobiera dane (transakcje) z pliku tekstowego. Następnie, dodaje do listy odpowiednią ilość transakcji. Blok jest dodawany wtedy, gdy liczba transakcji wynosi sześć. Na koniec, zwracana jest liczba bloków.

In [ ]:
def create_blockchain():
    file_path = "./transactions.txt"
    data = read_data(file_path)

    blockchain = []
    for index, data_blocks in enumerate(data):
        if len(data_blocks) == 6:
            if index == 0:
                blockchain.append(Block.genesis_block(data_blocks))
            else:
                blockchain.append(Block.add_block(blockchain[-1], data_blocks))
    return blockchain

Na koniec, zostaje stworzony blockchain na podstawie podanych transakcji. Następnie, jego zawartość zostaje wyświetlona oraz poprawność wszystkich bloków zostaje zweryfikowana. Każda zmiana w blokach powoduje zwrócenie przez funkcję "validating" wartości "False".

In [ ]:
# creating new blockchain
blockchain = create_blockchain()
# printing values of the blockchain
print_blockchain(blockchain)
print(validating(blockchain))

# testing validation of whole blockchain
blockchain[1].index = 3
print(validating(blockchain))

Genesis block ID:0 was added successfully!
Block ID:1 was added successfully!
Block <__main__.Block object at 0x106424a00>
{
	 index: 0
	 data: ['Ann Bob 0,59', 'Ann Bob 0,61', 'Ann Bob 0,62', 'Ann Bob 0,63', 'Ann Bob 0,64', 'Ann Bob 0,64']
	 timestamp: 1674034684.104717
	 previous_hash: 0000000000000000000000000000000000000000000000000000000000000000
	 nonce: 107
	 hash: 0023ffff670d24d533ea84da4c5e91df547eb16bc4d6690136ea029ed9277973
}

Block <__main__.Block object at 0x106426110>
{
	 index: 1
	 data: ['Ann Bob 0,65', 'Ann Bob 0,66', 'Ann Bob 0,67', 'Ann Bob 0,68', 'Ann Bob 0,69', 'Ann Bob 0,70']
	 timestamp: 1674034684.1053472
	 previous_hash: 0023ffff670d24d533ea84da4c5e91df547eb16bc4d6690136ea029ed9277973
	 nonce: 4696
	 hash: 0021fd431866e719d23527d006ea95cb8becd475036f0303943e375c20faf3a0
}

Blockchain is valid
Blockchain is not valid!
